*# Retail Sales Analysis - Data Cleaning*

In [59]:
import pandas as pd
import numpy as np

*# Load Data and Inspect*

In [62]:
df = pd.read_csv("C:/Users/gisel/Downloads/retail_store_sales(1).csv")
df

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False
...,...,...,...,...,...,...,...,...,...,...,...
12570,TXN_9347481,CUST_18,Patisserie,Item_23_PAT,38.0,4.0,152.0,Credit Card,In-store,2023-09-03,NaN
12571,TXN_4009414,CUST_03,Beverages,Item_2_BEV,6.5,9.0,58.5,Cash,Online,2022-08-12,False
12572,TXN_5306010,CUST_11,Butchers,Item_7_BUT,14.0,10.0,140.0,Cash,Online,2024-08-24,NaN
12573,TXN_5167298,CUST_04,Furniture,Item_7_FUR,14.0,6.0,84.0,Cash,Online,2023-12-30,True


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


*# Handle Missing Values*

In [66]:
# Fill missing 'Price Per Unit' based on matching 'Item'
price_map = df.set_index('Item')['Price Per Unit'].dropna().to_dict()
df['Price Per Unit'] = df['Price Per Unit'].fillna(df['Item'].map(price_map))

In [67]:
# Fill missing Quantity based on mean per price
mean_quantity = df.groupby('Price Per Unit')['Quantity'].mean()
df['Quantity'] = df['Quantity'].fillna(df['Price Per Unit'].map(mean_quantity))
df['Quantity'] = np.round(df['Quantity'])

In [68]:
# Fill missing 'Total Spent' where Quantity and Price are known
cond_total = df['Quantity'].notna() & df['Total Spent'].isna() & df['Price Per Unit'].notna()
df.loc[cond_total, 'Total Spent'] = df.loc[cond_total, 'Price Per Unit'] * df.loc[cond_total, 'Quantity']

In [69]:
# Fill 'Discount Applied' based on logic
def fill_discount(row):
    if pd.isna(row['Discount Applied']):
        return row['Total Spent'] != row['Price Per Unit'] * row['Quantity']
    return row['Discount Applied']

df['Discount Applied'] = df.apply(fill_discount, axis=1)

In [70]:
# Fill 'Item' using (Category + Price Per Unit)
lookup = df.dropna(subset=['Item']).drop_duplicates(subset=['Category', 'Price Per Unit'])
lookup = lookup.set_index(['Category', 'Price Per Unit'])['Item']

def fill_item(row):
    if pd.isna(row['Item']):
        return lookup.get((row['Category'], row['Price Per Unit']), row['Item'])
    return row['Item']

df['Item'] = df.apply(fill_item, axis=1)

*# Convert Data Types*

In [72]:
# Convert Quantity to integer
df['Quantity'] = df['Quantity'].astype(int)

*# Date Processing*

In [75]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], errors='coerce')

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    12575 non-null  object        
 1   Customer ID       12575 non-null  object        
 2   Category          12575 non-null  object        
 3   Item              12575 non-null  object        
 4   Price Per Unit    12575 non-null  float64       
 5   Quantity          12575 non-null  int32         
 6   Total Spent       12575 non-null  float64       
 7   Payment Method    12575 non-null  object        
 8   Location          12575 non-null  object        
 9   Transaction Date  12575 non-null  datetime64[ns]
 10  Discount Applied  12575 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int32(1), object(6)
memory usage: 945.7+ KB


In [89]:
df.head(20)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9,247.5,Credit Card,Online,2022-05-07,False
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7,87.5,Digital Wallet,Online,2022-10-02,False
5,TXN_7482416,CUST_09,Patisserie,Item_17_PAT,29.0,10,200.0,Credit Card,Online,2023-11-30,True
6,TXN_3652209,CUST_07,Food,Item_1_FOOD,5.0,8,40.0,Credit Card,In-store,2023-06-10,True
7,TXN_1372952,CUST_21,Furniture,Item_20_FUR,33.5,6,201.0,Digital Wallet,In-store,2024-04-02,True
8,TXN_9728486,CUST_23,Furniture,Item_16_FUR,27.5,1,27.5,Credit Card,In-store,2023-04-26,False
9,TXN_2722661,CUST_25,Butchers,Item_22_BUT,36.5,3,109.5,Cash,Online,2024-03-14,False
